# 模型结果比较

In [6]:
# 输出每个cell的运行时间
%load_ext autotime
# https://github.com/cpcloud/ipython-autotime
import sys
import pickle
import keras
import keras_contrib
import numpy as np

import warnings
warnings.filterwarnings("ignore")

sys.path.append('../')

word2idx = pickle.load(open('../saved_model_files/word2idx.pkl','rb'))
emb_matrix = pickle.load(open('../saved_model_files/emb_matrix.pkl','rb'))

# model_name = "bi_lstm_crf_4_3_2_39.h5"
model_name = "bi_lstm_crf_12_3_0_42.h5"
model = keras.models.load_model("../saved_model_files/" + model_name,
                                custom_objects={"CRF": keras_contrib.layers.CRF, "crf_loss": keras_contrib.losses.crf_loss,
                                                "crf_viterbi_accuracy": keras_contrib.metrics.crf_viterbi_accuracy})

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.09 s


In [7]:
from preprocess import DataProcessor
from evaluate import merge_preds4ner,f1_score4ner

WINDOW_SIZE = 70
EXPEND_SIZE = 10
BATCH_SIZE = 64

testset = pickle.load(open('../saved_model_files/testset.pkl', 'rb'))
testprocessor = DataProcessor(testset).data4NER(window=WINDOW_SIZE, pad=EXPEND_SIZE)
test_X, _ = testprocessor.get_ner_data()

preds = model.predict(test_X, batch_size=BATCH_SIZE, verbose=2)

pre_docs = merge_preds4ner(testset, testprocessor, preds)
source_docs = testset.docs

f1, prediction, recall, _ = f1_score4ner(pre_docs, source_docs, 'all')
print("【严格相交】F1:{:.4f}  P:{:.4f}  R:{:.4f}".format(f1, prediction, recall))

【严格相交】F1:0.7300  P:0.7200  R:0.7403
time: 19.6 s


## 参数不同时的结果对比

- bilstm-crf：[128,128]

    【严格相交】F1:0.7346  P:0.7150  R:0.7553

- bilstm-crf：[256]

    【严格相交】F1:0.7300  P:0.7200  R:0.7403